<a href="https://colab.research.google.com/github/TUstudents/gpHSP/blob/master/2_train_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
! rm -rf gpHSP
!git clone https://github.com/aspuru-guzik-group/gpHSP

Cloning into 'gpHSP'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 147 (delta 65), reused 79 (delta 35), pack-reused 38
Receiving objects: 100% (147/147), 32.98 MiB | 31.01 MiB/s, done.
Resolving deltas: 100% (79/79), done.


# Assuming in a colab enviroment

In [3]:
import sys
if 'google.colab' in sys.modules:
    print('In colab!')
    sys.path.insert(0,'gpHSP')
    !pip install ml_collections ngboost gpflow mordredcommunity[full] dill

In colab!
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 8.6 MB/s eta 0:00:00
  Created wheel for ml_collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94505 sha256=61959fda05c29d603c7e69d2509711b08bfce0bed9477baddd7a0b7570ed75de
  Stor

In [4]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import scipy.stats as stats

import ngboost
import tensorflow as tf
import gpflow as gpf
import gphsp

gphsp.notebook_context()
gphsp.print_modules([ngboost, tf , gpf])

ngboost    = 0.5.1
tensorflow = 2.15.0
gpflow     = 2.9.1


In [5]:
!cp -r -v /content/gdrive/MyDrive/gpHSP/data /content/gpHSP/

'/content/gdrive/MyDrive/gpHSP/data/sample_molecules.csv' -> '/content/gpHSP/data/sample_molecules.csv'
'/content/gdrive/MyDrive/gpHSP/data/Polymers_exp.csv' -> '/content/gpHSP/data/Polymers_exp.csv'
'/content/gdrive/MyDrive/gpHSP/data/Solvents_exp.csv' -> '/content/gpHSP/data/Solvents_exp.csv'
'/content/gdrive/MyDrive/gpHSP/data/mordred_features.npz' -> '/content/gpHSP/data/mordred_features.npz'
'/content/gdrive/MyDrive/gpHSP/data/Solvents_exp_REF.csv' -> '/content/gpHSP/data/Solvents_exp_REF.csv'
'/content/gdrive/MyDrive/gpHSP/data/HSPiP.csv' -> '/content/gpHSP/data/HSPiP.csv'


In [6]:
!cp -r -v /content/gdrive/MyDrive/gpHSP/models /content/gpHSP/

'/content/gdrive/MyDrive/gpHSP/models/δd_HSP_ngboost_mol.pkl' -> '/content/gpHSP/models/δd_HSP_ngboost_mol.pkl'
'/content/gdrive/MyDrive/gpHSP/models/δd_ngboost_polymer.pkl' -> '/content/gpHSP/models/δd_ngboost_polymer.pkl'
'/content/gdrive/MyDrive/gpHSP/models/δh_HSP_ngboost_mol.pkl' -> '/content/gpHSP/models/δh_HSP_ngboost_mol.pkl'
'/content/gdrive/MyDrive/gpHSP/models/δh_ngboost_polymer.pkl' -> '/content/gpHSP/models/δh_ngboost_polymer.pkl'
'/content/gdrive/MyDrive/gpHSP/models/δp_HSP_ngboost_mol.pkl' -> '/content/gpHSP/models/δp_HSP_ngboost_mol.pkl'
'/content/gdrive/MyDrive/gpHSP/models/δp_ngboost_polymer.pkl' -> '/content/gpHSP/models/δp_ngboost_polymer.pkl'


## Utilities

In [7]:
at_data_dir = lambda x: os.path.join('gpHSP/data', x)
at_model_dir = lambda x: os.path.join('gpHSP/models', x)

# Load data and get all smiles

In [15]:
df = pd.read_csv(at_data_dir('Solvents_exp.csv'))
df['smiles'] = df['smiles'].apply(gphsp.get_isomeric_smiles)
gphsp.peek_df(df)

Index(['key', 'Type', 'δd', 'δp', 'δh', 'smiles', 'ID_type', 'ID', 'Ref',
       'organic', 'n_electrons', 'n_atoms', 'charge', 'MolWt', 'label',
       'finished', 'job_name', 'homo', 'lumo', 'gap', 'd-moments', 'dipole',
       'polar', 'run_time', 'n_cores', 'compute_time', 'Area', 'Hba', 'Hbd',
       'Volume', 'sigma_mom_0', 'sigma_mom_1', 'sigma_mom_2', 'sigma_mom_3',
       'sigma_mom_4', 'sigma_mom_5', 'sigma_norm', 'sigma_profile', 'drug'],
      dtype='object')
(193, 39)


,key,Type,δd,δp,δh,smiles,ID_type,ID,Ref,organic,...,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile,drug
0,"1,1,1-Trichloroethane",Solvent,16.8,4.3,2.0,CC(Cl)(Cl)Cl,CAS,71-55-6,1,True,...,0.126,0,16.6,-3.836,4.24,-2.021,1.54,41.428,[ 0. 0. 0. 0. ...,False


In [16]:
poly_df = pd.read_csv(at_data_dir('Polymers_exp.csv'))
poly_df['smiles'] = poly_df['smiles'].apply(gphsp.get_isomeric_smiles)
gphsp.peek_df(poly_df)

Index(['label', 'δd', 'δp', 'δh', 'smiles', 'test', 'poly_label', 'n_copies',
       'organic', 'n_electrons', 'n_atoms', 'charge', 'MolWt', 'n_frags',
       'largest', 'finished', 'job_name', 'poly_smiles', 'homo', 'lumo', 'gap',
       'd-moments', 'dipole', 'polar', 'run_time', 'n_cores', 'compute_time',
       'Area', 'Hba', 'Hbd', 'Volume', 'sigma_mom_0', 'sigma_mom_1',
       'sigma_mom_2', 'sigma_mom_3', 'sigma_mom_4', 'sigma_mom_5',
       'sigma_norm', 'sigma_profile'],
      dtype='object')
(31, 39)


,label,δd,δp,δh,smiles,test,poly_label,n_copies,organic,n_electrons,...,Hbd,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile
0,Polyacrylonitrile-n5,20.0,15.1,7.9,CCC#N,True,Polyacrylonitrile,5,True,110,...,0.22,0.343,0,175.0,26.1,154.0,51.4,165.0,66.77,[ 0. 0. 0. 0. ...


Load features

In [17]:
features = gphsp.SmilesMap(at_data_dir('mordred_features.npz'))

# Train rfHSP model on single molecules

In [18]:
save_model = True
model_suffix = 'HSP_ngboost_mol.pkl'

models = {}
x = features(df['smiles'].to_numpy(str))
names = gphsp.Y_COLS
print(names)
for name in tqdm(names, total=len(names)):
    y = df[name].to_numpy(np.float32).reshape(-1,1)
    model = ngboost.NGBRegressor().fit(x, y)
    y_dists = model.pred_dist(x)
    models[name] = model
    if save_model:
        fname = at_model_dir(f"{name}_{model_suffix}")
        gphsp.save_model(model, fname)

['δd', 'δp', 'δh']


  0%|          | 0/3 [00:00<?, ?it/s]

[iter 0] loss=2.0292 val_loss=0.0000 scale=1.0000 norm=1.5771


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[iter 100] loss=1.3892 val_loss=0.0000 scale=2.0000 norm=1.7636
[iter 200] loss=0.7992 val_loss=0.0000 scale=2.0000 norm=1.2279
[iter 300] loss=0.3836 val_loss=0.0000 scale=2.0000 norm=0.9976
[iter 400] loss=0.0842 val_loss=0.0000 scale=1.0000 norm=0.4317


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[iter 0] loss=3.0726 val_loss=0.0000 scale=1.0000 norm=4.1787
[iter 100] loss=2.5094 val_loss=0.0000 scale=2.0000 norm=4.7659
[iter 200] loss=1.9741 val_loss=0.0000 scale=1.0000 norm=1.4607
[iter 300] loss=1.4956 val_loss=0.0000 scale=2.0000 norm=2.1714
[iter 400] loss=1.1834 val_loss=0.0000 scale=2.0000 norm=1.7959


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[iter 0] loss=3.1769 val_loss=0.0000 scale=1.0000 norm=4.6890
[iter 100] loss=2.3695 val_loss=0.0000 scale=2.0000 norm=3.6644
[iter 200] loss=1.6093 val_loss=0.0000 scale=2.0000 norm=2.0204
[iter 300] loss=0.9949 val_loss=0.0000 scale=2.0000 norm=1.5120
[iter 400] loss=0.6177 val_loss=0.0000 scale=1.0000 norm=0.6421


# Train gpHSP model on polymers

In [19]:
model_suffix = 'HSP_ngboost_mol.pkl'
names = gphsp.Y_COLS
models = {name: gphsp.load_model(at_model_dir(f"{name}_{model_suffix}")) for name in names}
mol_x = features(poly_df['smiles'].to_numpy(str))
x = gphsp.predictions_as_features(mol_x, models)

In [20]:
model_suffix = 'ngboost_polymer.pkl'

save_models = True
poly_models = {}
parts = {'mean':[0,1, 2], 'std':[3, 4, 5]}
for name in tqdm(names, total=len(names)):
    y = poly_df[name].to_numpy(np.float32).reshape(-1,1)
    model = ngboost.NGBRegressor().fit(x, y)
    y_dists = model.pred_dist(x)
    poly_models[name] = model
    gpf.utilities.print_summary(model)
    if save_models:
        adir = at_model_dir(f'{name}_{model_suffix}')
        gphsp.save_model(model, adir)

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[iter 0] loss=1.2875 val_loss=0.0000 scale=1.0000 norm=0.9061
[iter 100] loss=0.6297 val_loss=0.0000 scale=2.0000 norm=1.0949
[iter 200] loss=-0.1828 val_loss=0.0000 scale=2.0000 norm=0.8715
[iter 300] loss=-0.9335 val_loss=0.0000 scale=2.0000 norm=0.7853
[iter 400] loss=-1.5667 val_loss=0.0000 scale=2.0000 norm=0.7064


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[iter 0] loss=2.8508 val_loss=0.0000 scale=1.0000 norm=3.5836
[iter 100] loss=1.9128 val_loss=0.0000 scale=2.0000 norm=2.3185
[iter 200] loss=1.1392 val_loss=0.0000 scale=2.0000 norm=1.3812
[iter 300] loss=0.4330 val_loss=0.0000 scale=2.0000 norm=1.0658
[iter 400] loss=-0.2261 val_loss=0.0000 scale=2.0000 norm=0.9162


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[iter 0] loss=2.8877 val_loss=0.0000 scale=1.0000 norm=3.3176
[iter 100] loss=1.8928 val_loss=0.0000 scale=2.0000 norm=2.4242
[iter 200] loss=1.1496 val_loss=0.0000 scale=2.0000 norm=1.5382
[iter 300] loss=0.4484 val_loss=0.0000 scale=2.0000 norm=1.2158
[iter 400] loss=-0.2017 val_loss=0.0000 scale=2.0000 norm=1.0551


In [21]:
!cp -r -v /content/gpHSP/models /content/gdrive/MyDrive/gpHSP/

'/content/gpHSP/models/.ipynb_checkpoints' -> '/content/gdrive/MyDrive/gpHSP/models/.ipynb_checkpoints'
'/content/gpHSP/models/δd_HSP_ngboost_mol.pkl' -> '/content/gdrive/MyDrive/gpHSP/models/δd_HSP_ngboost_mol.pkl'
'/content/gpHSP/models/δp_HSP_ngboost_mol.pkl' -> '/content/gdrive/MyDrive/gpHSP/models/δp_HSP_ngboost_mol.pkl'
'/content/gpHSP/models/δh_HSP_ngboost_mol.pkl' -> '/content/gdrive/MyDrive/gpHSP/models/δh_HSP_ngboost_mol.pkl'
'/content/gpHSP/models/δd_ngboost_polymer.pkl' -> '/content/gdrive/MyDrive/gpHSP/models/δd_ngboost_polymer.pkl'
'/content/gpHSP/models/δp_ngboost_polymer.pkl' -> '/content/gdrive/MyDrive/gpHSP/models/δp_ngboost_polymer.pkl'
'/content/gpHSP/models/δh_ngboost_polymer.pkl' -> '/content/gdrive/MyDrive/gpHSP/models/δh_ngboost_polymer.pkl'
